# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
#Define a workspace
ws = Workspace.from_config()

#Create an experiment
exp = Experiment(workspace=ws, name="Hyperparameter_Tuning")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()


Workspace name: quick-starts-ws-243971
Azure region: westus2
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-243971


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProjectml"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-11-12T16:38:33.025000+00:00', 'errors': None, 'creationTime': '2023-11-12T16:38:25.206531+00:00', 'modifiedTime': '2023-11-12T16:38:35.267266+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
import pandas as pd
pd_path='https://raw.githubusercontent.com/AnnaDM87/Udacity_CAPSTONE/main/starter_file/Employee.csv'
df = pd.read_csv(pd_path)

df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [5]:
import os
os.listdir()

['.ipynb_checkpoints',
 'automl.ipynb',
 'conda_dependencies.yml',
 'hyperparameter_tuning.ipynb',
 'outputs',
 'train.py',
 'Users']

In [6]:
from train import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

#Use the clean_data function to clean your dataset.
x, y = clean_data(df)
x.head()

#Split the dataset into train and test dataset. Combine x_train and y_train. 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
df_train = pd.concat([x_train,y_train], axis=1)
df_test = pd.concat([x_test,y_test], axis=1)

#Convert x_train and y_train (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path_train = 'data/train.csv'
path_test = 'data/test.csv'
df_train.to_csv(path_train)
df_test.to_csv(path_test)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/test.csv'))])
print("Successfully converted the dataset to TabularDataset format.")

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading data/test.csv
Uploaded data/test.csv, 1 files out of an estimated total of 2
Uploading data/train.csv
Uploaded data/train.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Successfully converted the dataset to TabularDataset format.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
project_folder = './automl-proj'

In [8]:

from azureml.train.automl import AutoMLConfig
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=cpu_cluster,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="LeaveOrNot",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config,show_output=True)

Submitting remote run.
No run_configuration provided, running on AzureProjectml with default configuration
Running on remote compute: AzureProjectml


Experiment,Id,Type,Status,Details Page,Docs Page
Hyperparameter_Tuning,AutoML_72372a07-3491-44ad-8094-78dfb6b86645,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails

#Launch the widget to view the progress and results
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve and save your best automl model.

best_run, best_model=remote_run.get_output()
print(best_run)
print(best_model)



Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: Hyperparameter_Tuning,
Id: AutoML_72372a07-3491-44ad-8094-78dfb6b86645_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('5', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alph

In [14]:
print(best_run)

Run(Experiment: Hyperparameter_Tuning,
Id: AutoML_72372a07-3491-44ad-8094-78dfb6b86645_38,
Type: azureml.scriptrun,
Status: Completed)


In [15]:
print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('5', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=0.625, subsample=0.8, tree_method='auto'))], verbose=False)), ('18', Pipeline(memory=None, steps=[('standardscalerwrapp

In [12]:
print("Best run metrics :", best_run.get_metrics())
print("Best accuracy run metrics ",best_run.get_metrics(name='accuracy'))
print("Best run details :", best_run.get_details())

Best run metrics : {'norm_macro_recall': 0.5999285150654075, 'AUC_micro': 0.9115810484603136, 'recall_score_macro': 0.7999642575327037, 'balanced_accuracy': 0.7999642575327037, 'recall_score_micro': 0.8460500550197985, 'average_precision_score_micro': 0.9101229227129871, 'AUC_macro': 0.8756426181668275, 'average_precision_score_macro': 0.8818872465881098, 'matthews_correlation': 0.6488521425178014, 'f1_score_macro': 0.8166453732493683, 'accuracy': 0.8460500550197985, 'AUC_weighted': 0.8756426181668275, 'precision_score_micro': 0.8460500550197985, 'precision_score_macro': 0.8510459238362263, 'weighted_accuracy': 0.8833507209739064, 'log_loss': 0.38050272890987796, 'f1_score_weighted': 0.8401513596816903, 'recall_score_weighted': 0.8460500550197985, 'precision_score_weighted': 0.8482811664092994, 'average_precision_score_weighted': 0.891311529506826, 'f1_score_micro': 0.8460500550197985, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_72372a07-3491-44ad-8094-78dfb6b86645_

In [33]:
#TODO: Save the best model
best_run.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/12d42c59/classes.interpret.json',
 'explanation/12d42c59/eval_data_viz.interpret.json',
 'explanation/12d42c59/expected_values.interpret.json',
 'explanation/12d42c59/features.interpret.json',
 'explanation/12d42c59/global_names/0.interpret.json',
 'explanation/12d42c59/global_rank/0.interpret.json',
 'explanation/12d42c59/global_values/0.interpret.json',
 'explanation/12d42c59/local_importance_values.interpret.json',
 'explanation/12d42c59/per_class_names/0.interpret.json',
 'explanation/12d42c59/per_class_rank/0.interpret.json',
 'explanation/12d42c59/per_class_values/0.interpret.json',
 'explanation/12d42c59/rich_metadata.interpret.json',
 'explanation/12d42c59/true_ys_viz.interpret.json',
 'explanation/12d42c59/visualization_dict.interpret.json',
 'explanation/12d42c59/ys_pred_proba_viz.interpret.json',
 'explanation/12d42c59/ys_pred_viz.interpret.json',
 'explanation/2f1cefba/classes.interpret.json',
 'expl

In [13]:
import joblib
from azureml.core.model import Model

#Save the best model
os.makedirs('results', exist_ok=True)
joblib.dump(best_model, filename="results/automl_model.pkl")
model = remote_run.register_model(model_name=best_run.properties['model_name'], description='Best AutoML model')
print("Model saved successfully")

Model saved successfully


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
